In [1]:
print("hello world")

print("hello vish")

hello world
hello vish


# CUDA Setup

In [2]:
# Checking cuda version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install nvcc4jupyter # run cuda from jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp5qrwbxjy".


# Download MNIST

In [35]:
!wget -q http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

!gunzip *.gz
!pwd


gzip: *.gz: No such file or directory
/content


## Demo CUDA program

In [4]:
  %%cuda
#include <cstdio>
#include <iostream>

using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
    int block = 256 * blockIdx.x;
    int max = 0;

    for (int i = block; i < min(256 + block, n); i++) {

        if (max < a[i]) {
            max = a[i];
        }
    }
    b[blockIdx.x] = max;
}

int main()
{

    int n;
    n = 10;
    int a[n];

    for (int i = 0; i < n; i++) {
        a[i] = rand() % n;
        cout << a[i] << "\t";
    }

    cout << endl;

    cudaEvent_t start, end;
    int *ad, *bd;
    int size = n * sizeof(int);
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    int grids = ceil(n * 1.0f / 256.0f);
    cudaMalloc(&bd, grids * sizeof(int));

    dim3 grid(grids, 1);
    dim3 block(1, 1);

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    while (n > 1) {
        maxi<<<grids, block>>>(ad, bd, n);
        n = ceil(n * 1.0f / 256.0f);
        cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    int ans[2];
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

    cout << "The maximum element is : " << ans[0] << endl;

    cout << "The time required : ";
    cout << time << endl;
}

3	6	7	5	3	5	6	2	9	1	
The maximum element is : 9
The time required : 117.453



In [6]:

# This is a program that takes in a randomly initialized 30 x 30 array and outputs
# a 10 x 10 array where each element is the average of every 3 x 3 non-overlapping
# window in the 30 x 30 array

%%cuda
#include <cstdio>
#include <iostream>

using namespace std;

__global__ void averageMatrixWindows(float** input, float** output) {

}

int main() {
  int size = 30;
  int window_size = 3;

  float* inp_matr[size];
  for (int i = 0; i < size; i++) {
    inp_matr[i] = new float[size];
    for (int j = 0; j < size; j++)
      inp_matr[i][j] = rand() % size;
  }

  float* out_matr[size / window_size];
  for (int i = 0; i < size / window_size; i++)
    out_matr[i] = new float[size / window_size];


  return 0;
}

In [103]:
%%cuda
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <curand.h>
#include <curand_kernel.h>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <cassert>
#include <cuda_runtime.h>
#include <vector>
#include <fstream>
#include <cstdint>


void readMNISTImages(const std::string &filename, float *images, int &training_size, int &rows, int &cols) {
    std::ifstream file(filename, std::ios::binary);

    // Read the header
    int magic_number = 0, num_images = 0;
    file.read(reinterpret_cast<char *>(&magic_number), 4);
    file.read(reinterpret_cast<char *>(&num_images), 4);
    file.read(reinterpret_cast<char *>(&rows), 4);
    file.read(reinterpret_cast<char *>(&cols), 4);

    // Convert from big-endian to little-endian
    magic_number = __builtin_bswap32(magic_number);
    num_images = __builtin_bswap32(num_images);
    rows = __builtin_bswap32(rows);
    cols = __builtin_bswap32(cols);

    // Ensure training_size does not exceed num_images in the dataset
    if (training_size > num_images) {
        std::cerr << "Requested training size exceeds number of available images.\n";
        training_size = num_images;
    }

    // Compute size of one image
    int image_size = rows * cols;

    // Read the required number of images into the array
    for (int i = 0; i < training_size; ++i) {
        std::vector<uint8_t> buffer(image_size);
        file.read(reinterpret_cast<char *>(buffer.data()), image_size);
        for (int j = 0; j < image_size; ++j) {
            images[i * image_size + j] = buffer[j] / 255.0f; // Normalize to [0, 1]
        }
    }

    file.close();
}

void readMNISTLabels(const std::string &filename, float *labels, int training_size, int num_classes) {
    std::ifstream file(filename, std::ios::binary);
    if (!file.is_open()) {
        std::cerr << "Cannot open file: " << filename << "\n";
        return;
    }

    // Read the header
    int32_t magic_number = 0, num_labels = 0;
    file.read(reinterpret_cast<char *>(&magic_number), 4);
    file.read(reinterpret_cast<char *>(&num_labels), 4);

    // Convert from big-endian to little-endian
    magic_number = __builtin_bswap32(magic_number);
    num_labels = __builtin_bswap32(num_labels);

    // Ensure training_size does not exceed num_labels in the dataset
    if (training_size > num_labels) {
        std::cerr << "Requested training size exceeds number of available labels.\n";
        training_size = num_labels;
    }

    for (int i = 0; i < training_size * num_classes; i++) {
      labels[i] = 0.0f;
    }

    // Read the labels and convert to one-hot
    for (int i = 0; i < training_size; ++i) {
        unsigned char label = 0;
        file.read(reinterpret_cast<char *>(&label), 1);
        labels[i * num_classes + static_cast<int>(label)] = 1.0f; // Set the one-hot value
    }
}



// Error checking macro
#define CUDA_CALL(x) do { if((x)!=cudaSuccess) { \
    fprintf(stderr,"Error at %s:%d: %s\n",__FILE__,__LINE__,cudaGetErrorString(x)); \
    exit(EXIT_FAILURE);}} while(0)

// Indexing helpers
inline __device__ __host__ int idx4(int n, int h, int w, int c, int H, int W, int C) {
    return (((n * H) + h) * W + w) * C + c;
}

inline __device__ __host__ int idx3(int n, int h, int w, int H, int W) {
    return (n*H*W + h*W + w);
}

// ---------------------------------------------
// GPU kernels (Forward) [Same as before]
// ---------------------------------------------
__global__ void convForwardKernel(const float* __restrict__ input,
                                  const float* __restrict__ kernel,
                                  float* __restrict__ output,
                                  int N, int H, int W, int C_in,
                                  int K, int C_out) {
    int n = blockIdx.x;
    int h = threadIdx.y;
    int w = threadIdx.x;
    for (int co = 0; co < C_out; co++) {
        float val = 0.0f;
        for (int ci = 0; ci < C_in; ci++) {
            for (int kh = 0; kh < K; kh++) {
                for (int kw = 0; kw < K; kw++) {
                    float iv = input[idx4(n, h+kh, w+kw, ci, H, W, C_in)];
                    float kv = kernel[((ci * K + kh)*K + kw)*C_out + co];
                    val += iv * kv;
                }
            }
        }
        output[idx4(n, h, w, co, H - K + 1, W - K + 1, C_out)] = val;
    }
}

__global__ void maxPoolForwardKernel(const float* __restrict__ input,
                                     float* __restrict__ output,
                                     int* __restrict__ max_idx, // store argmax indices
                                     int N, int H_in, int W_in, int C,
                                     int stride) {
    int n = blockIdx.x;
    int oh = threadIdx.y;
    int ow = threadIdx.x;
    int hstart = oh * stride;
    int wstart = ow * stride;
    for (int c = 0; c < C; c++) {
        float max_val = -1e10;
        int max_pos = 0;
        for (int i = 0; i < stride; i++) {
            for (int j = 0; j < stride; j++) {
                int in_idx = idx4(n,hstart+i,wstart+j,c,H_in,W_in,C);
                float val = input[in_idx];
                if (val > max_val) {
                    max_val = val;
                    max_pos = in_idx;
                }
            }
        }
        int out_idx = idx4(n, oh, ow, c, H_in/stride, W_in/stride, C);
        output[out_idx] = max_val;
        max_idx[out_idx] = max_pos; // store where we took max from
    }
}

__global__ void fcForwardKernel(const float* __restrict__ input,
                                const float* __restrict__ w,
                                const float* __restrict__ b,
                                float* __restrict__ output,
                                int N, int D_in, int D_out) {
    int n = blockIdx.x;
    for (int d = 0; d < D_out; d++) {
        float val = b[d];
        for (int i = 0; i < D_in; i++) {
            val += input[n*D_in + i]*w[i*D_out + d];
        }
        output[n*D_out + d] = val;
    }
}

__global__ void softmaxKernel(float* data, int N, int D) {
    int n = blockIdx.x;
    float maxval = -1e10;
    for (int d = 0; d < D; d++) {
        float v = data[n*D + d];
        if (v > maxval) maxval = v;
    }
    float sum = 0.0f;
    for (int d = 0; d < D; d++) {
        float expv = expf(data[n*D + d]-maxval);
        data[n*D + d] = expv;
        sum += expv;
    }
    for (int d = 0; d < D; d++) {
        data[n*D + d] /= sum;
    }
}

// ---------------------------------------------
// GPU kernels (Backward)
// ---------------------------------------------

// Conv Backward Kernel
// dOut: (N, H_out, W_out, C_out)
// input: (N, H_in, W_in, C_in)
// kernel: (C_in, K, K, C_out)
//
// dIn: gradient w.r.t input
// dW: gradient w.r.t weights
__global__ void convBackwardKernel(const float* __restrict__ input,
                                   const float* __restrict__ dOut,
                                   float* __restrict__ dIn,
                                   float* __restrict__ dW,
                                   int N, int H, int W, int C_in,
                                   int K, int C_out) {
    // We can have multiple kernels: one for dIn and one for dW for simplicity.
    // dIn computation:
    // Each thread: one element of input gradient
    // A direct approach is expensive; consider reversed conv or partial expansions.
    // For simplicity, we do a naive approach:

    int n = blockIdx.x;
    // We'll do a parallel reduction per output pixel contributing to a given input pixel
    // This is naive and may be slow for large scale.
    for (int h_in=0; h_in<H; h_in++) {
        for (int w_in=0; w_in<W; w_in++) {
            for (int c_in=0; c_in<C_in; c_in++) {
                float grad_val=0.0f;
                // sum over all c_out, kh, kw
                for(int c_out=0;c_out<C_out;c_out++){
                    for(int kh=0;kh<K;kh++){
                        for(int kw=0;kw<K;kw++){
                            int h_out = h_in - kh;
                            int w_out = w_in - kw;
                            if (h_out>=0 && w_out>=0 && h_out< (H-K+1) && w_out< (W-K+1)){
                                float dout_val = dOut[idx4(n,h_out,w_out,c_out,H-K+1,W-K+1,C_out)];
                                float w_val = dW[((c_in*K+kh)*K+kw)*C_out + c_out];
                                grad_val += dout_val * w_val;
                            }
                        }
                    }
                }
                dIn[idx4(n,h_in,w_in,c_in,H,W,C_in)] = grad_val;
            }
        }
    }
}

__global__ void convBackwardWeightKernel(const float* __restrict__ input,
                                         const float* __restrict__ dOut,
                                         float* __restrict__ dW,
                                         int N, int H, int W, int C_in,
                                         int K, int C_out) {
    // accumulate gradients for weights
    // w_grad shape = (C_in,K,K,C_out)
    // Let's do a simple approach: one thread per weight
    int c_in = threadIdx.x;
    int kh   = threadIdx.y;
    int kw   = threadIdx.z;
    int c_out = blockIdx.x;

    float grad = 0.0f;
    for (int n=0;n<N;n++){
        for (int h_out=0;h_out<H-K+1;h_out++){
            for (int w_out=0;w_out<W-K+1;w_out++){
                float val_in = input[idx4(n, h_out+kh, w_out+kw, c_in, H, W, C_in)];
                float val_dout = dOut[idx4(n,h_out,w_out,c_out,H-K+1,W-K+1,C_out)];
                grad += val_in*val_dout;
            }
        }
    }
    dW[((c_in*K+kh)*K+kw)*C_out + c_out] += grad;
}

// MaxPool Backward Kernel
__global__ void maxPoolBackwardKernel(const float* __restrict__ dOut,
                                      float* __restrict__ dIn,
                                      const int* __restrict__ max_idx,
                                      int N, int H_in, int W_in, int C,
                                      int stride) {
    int n = blockIdx.x;
    int oh = threadIdx.y;
    int ow = threadIdx.x;
    for (int c=0;c<C;c++){
        int out_idx = idx4(n,oh,ow,c,H_in/stride,W_in/stride,C);
        float grad_val = dOut[out_idx];
        int max_pos = max_idx[out_idx];
        // Only the max position gets the gradient
        atomicAdd(&dIn[max_pos], grad_val);
    }
}

// FC Backward Kernels
__global__ void fcBackwardKernel(const float* __restrict__ input,
                                 const float* __restrict__ dOut,
                                 float* __restrict__ dIn,
                                 float* __restrict__ dW,
                                 float* __restrict__ dB,
                                 int N, int D_in, int D_out) {
    int n = blockIdx.x;
    // Compute gradients w.r.t input
    for (int d=0;d<D_in;d++){
        float grad_val=0.0f;
        for (int d_o=0;d_o<D_out;d_o++){
            float dw = dW[d*D_out + d_o]; // w used for grad w.r.t in?
            // Actually we should use original weights (not gradient), so we might need original w.
            // Let's assume we have original weights in global memory accessible.
            // In this kernel we just accumulate dW and dB.
        }
    }
    // Actually, better approach: separate kernels for clarity.

    // dB gradient
    for (int d_o=0; d_o<D_out; d_o++){
        atomicAdd(&dB[d_o], dOut[n*D_out + d_o]);
    }

    // dW gradient
    for (int d_o=0; d_o<D_out; d_o++){
        float grad_val = dOut[n*D_out + d_o];
        for (int d_i=0; d_i<D_in; d_i++){
            atomicAdd(&dW[d_i*D_out + d_o], input[n*D_in + d_i]*grad_val);
        }
    }
}

// FC dIn kernel
__global__ void fcBackwardInputKernel(const float* __restrict__ dOut,
                                      const float* __restrict__ w,
                                      float* __restrict__ dIn,
                                      int N, int D_in, int D_out) {
    int n = blockIdx.x;
    for (int d_i=0; d_i<D_in; d_i++){
        float grad_val=0.0f;
        for (int d_o=0; d_o<D_out; d_o++){
            grad_val += dOut[n*D_out + d_o]*w[d_i*D_out + d_o];
        }
        dIn[n*D_in + d_i] = grad_val;
    }
}

// Flatten backward kernel (just copy)
__global__ void flattenBackwardKernel(const float* dOut, float* dIn, int N, int H, int W, int C) {
    int n = blockIdx.x;
    int size = H*W*C;
    for (int i=0;i<size;i++){
        dIn[n*size + i] = dOut[n*size + i];
    }
}

// Softmax backward is usually combined with cross-entropy loss.
// For demonstration, we assume dOut is provided from outside.

// ---------------------------------------------
// Base Layer Class
// ---------------------------------------------
class Layer {
public:
    virtual ~Layer() {}
    bool isConvOrPool = false;

    // Store forward input/output for backward
    float *input_cache = nullptr;
    float *output_cache = nullptr;

    // Output shape after forward
    int outN, outH, outW, outC, outD;

    virtual float* forward(float* input, int N, int H, int W, int C) {
        std::cerr << "Forward (N,H,W,C) not implemented.\n";
        return nullptr;
    }
    virtual float* forward(float* input, int N, int D) {
        std::cerr << "Forward (N,D) not implemented.\n";
        return nullptr;
    }

    // backward takes dOut and returns dIn
    // For conv/pool: dOut is (N,outH,outW,outC), returns dIn (N,H,W,C)
    // For FC: dOut is (N,D_out), returns dIn (N,D_in)
    // For flatten: dOut is (N,D), returns dIn (N,H,W,C)
    virtual float* backward(float* dOut) {
        std::cerr << "Backward not implemented.\n";
        return nullptr;
    }

    // For layers with parameters, store gradients here and implement parameter update later.
    virtual void zeroGrad() {}
    virtual void updateParams(float lr) {}
};

// ---------------------------------------------
// Conv2D Layer
// ---------------------------------------------
class Conv2D : public Layer {
public:
    int window_size, channels, neurons;
    float *d_kernel = nullptr;

    // For backprop
    float *d_kernel_grad = nullptr;
    int inH, inW, inC, inN;

    Conv2D(int window_size_, int channels_, int neurons_):
        window_size(window_size_), channels(channels_), neurons(neurons_) {
        int k_size = channels*window_size*window_size*neurons;
        float *h_kernel = (float*)malloc(k_size*sizeof(float));
        for (int i = 0; i < k_size; i++)
            h_kernel[i] = (float)rand()/RAND_MAX;
        CUDA_CALL(cudaMalloc(&d_kernel, k_size*sizeof(float)));
        CUDA_CALL(cudaMemcpy(d_kernel, h_kernel, k_size*sizeof(float), cudaMemcpyHostToDevice));
        free(h_kernel);

        CUDA_CALL(cudaMalloc(&d_kernel_grad, k_size*sizeof(float)));
        CUDA_CALL(cudaMemset(d_kernel_grad, 0, k_size*sizeof(float)));

        isConvOrPool = true;
    }

    ~Conv2D() {
        if (d_kernel) CUDA_CALL(cudaFree(d_kernel));
        if (d_kernel_grad) CUDA_CALL(cudaFree(d_kernel_grad));
        // if (input_cache) CUDA_CALL(cudaFree(input_cache));
        if (output_cache) CUDA_CALL(cudaFree(output_cache));
    }

    float* forward(float* input, int N, int H, int W, int C) override {
        inN=N; inH=H; inW=W; inC=C;
        int outH = H - window_size + 1;
        int outW = W - window_size + 1;
        int outC = neurons;

        // Store input for backward
        input_cache = input; // we assume input is from previous layer (already on GPU)
        float *d_output;
        CUDA_CALL(cudaMalloc(&d_output, N*outH*outW*outC*sizeof(float)));

        dim3 blockDim(outW, outH);
        dim3 gridDim(N);
        convForwardKernel<<<gridDim, blockDim>>>(input, d_kernel, d_output, N, H, W, C, window_size, neurons);
        CUDA_CALL(cudaDeviceSynchronize());

        output_cache = d_output;
        this->outN = N; this->outH = outH; this->outW = outW; this->outC = outC;
        return d_output;
    }

    float* backward(float* dOut) override {
        // dOut: N,outH,outW,outC
        // Compute dIn and dW
        float *dIn;
        CUDA_CALL(cudaMalloc(&dIn, inN*inH*inW*inC*sizeof(float)));
        CUDA_CALL(cudaMemset(dIn,0,inN*inH*inW*inC*sizeof(float)));

        // zeroGrad for kernel
        CUDA_CALL(cudaMemset(d_kernel_grad, 0, channels*window_size*window_size*neurons*sizeof(float)));

        // Compute dIn & dW
        {
            dim3 grid(inN);
            convBackwardKernel<<<grid,1>>>(input_cache, dOut, dIn, d_kernel, inN,inH,inW,inC,window_size,neurons);
        }

        {
            dim3 gridW(neurons);
            dim3 blockW(channels, window_size, window_size);
            convBackwardWeightKernel<<<gridW, blockW>>>(input_cache, dOut, d_kernel_grad, inN, inH, inW, inC, window_size, neurons);
        }

        CUDA_CALL(cudaDeviceSynchronize());

        return dIn;
    }

    void zeroGrad() override {
        CUDA_CALL(cudaMemset(d_kernel_grad,0,channels*window_size*window_size*neurons*sizeof(float)));
    }

    void updateParams(float lr) override {
        // W = W - lr*dW
        int size = channels*window_size*window_size*neurons;
        thrust::device_ptr<float> w_ptr(d_kernel);
        thrust::device_ptr<float> dw_ptr(d_kernel_grad);
        for (int i=0;i<size;i++){
            w_ptr[i] -= lr*dw_ptr[i];
        }
    }
};

// ---------------------------------------------
// MaxPool Layer
// ---------------------------------------------
class MaxPool : public Layer {
public:
    int stride;
    int inH,inW,inC,inN;
    int *d_max_idx=nullptr;

    MaxPool(int stride_): stride(stride_) {
        isConvOrPool = true;
    }

    ~MaxPool() {
        if (d_max_idx) CUDA_CALL(cudaFree(d_max_idx));
        // if (input_cache && input_cache!=output_cache) CUDA_CALL(cudaFree(input_cache));
        if (output_cache) CUDA_CALL(cudaFree(output_cache));
    }

    float* forward(float* input, int N, int H, int W, int C) override {
        inN=N; inH=H; inW=W; inC=C;
        int outH = H/stride;
        int outW = W/stride;
        int outC = C;
        float *d_output;
        CUDA_CALL(cudaMalloc(&d_output, N*outH*outW*C*sizeof(float)));
        CUDA_CALL(cudaMalloc(&d_max_idx, N*outH*outW*C*sizeof(int)));

        dim3 blockDim(outW, outH);
        dim3 gridDim(N);
        maxPoolForwardKernel<<<gridDim, blockDim>>>(input, d_output, d_max_idx, N, H, W, C, stride);
        CUDA_CALL(cudaDeviceSynchronize());

        input_cache = input;
        output_cache = d_output;
        this->outN=N; this->outH=outH; this->outW=outW; this->outC=outC;
        return d_output;
    }

    float* backward(float* dOut) override {
        // dOut: N,outH,outW,outC
        // We know exactly where each output came from.
        float *dIn;
        CUDA_CALL(cudaMalloc(&dIn, inN*inH*inW*inC*sizeof(float)));
        CUDA_CALL(cudaMemset(dIn, 0, inN*inH*inW*inC*sizeof(float)));

        dim3 blockDim(outW, outH);
        dim3 gridDim(inN);
        maxPoolBackwardKernel<<<gridDim, blockDim>>>(dOut, dIn, d_max_idx, inN, inH, inW, inC, stride);
        CUDA_CALL(cudaDeviceSynchronize());
        return dIn;
    }
};

// ---------------------------------------------
// Flatten Layer
// ---------------------------------------------
class Flatten : public Layer {
public:
    int inN,inH,inW,inC;
    Flatten() {
        isConvOrPool = false;
    }

    float* forward(float* input, int N, int H, int W, int C) override {
        inN=N;inH=H;inW=W;inC=C;
        int D = H*W*C;
        float *d_output;
        CUDA_CALL(cudaMalloc(&d_output, N*D*sizeof(float)));
        // Flatten kernel
        for (int i=0;i<N;i++){
            // we can just do a cudaMemcpy for simplicity, since memory is contiguous
            // input and output have same indexing pattern for flatten.
            // Actually indexing matches (N,H,W,C) vs (N,D) but memory layout is contiguous anyway.
            CUDA_CALL(cudaMemcpy(d_output + i*D, input + i*D, D*sizeof(float), cudaMemcpyDeviceToDevice));
        }

        input_cache = input;
        output_cache = d_output;
        this->outN = N; this->outD = D; this->outH=0; this->outW=0; this->outC=0;
        return d_output;
    }

    float* backward(float* dOut) override {
        // dOut: (N,D)
        // dIn: (N,H,W,C)
        int D = inH*inW*inC;
        float *dIn;
        CUDA_CALL(cudaMalloc(&dIn, inN*D*sizeof(float)));
        // Just copy back
        for (int i=0;i<inN;i++){
            CUDA_CALL(cudaMemcpy(dIn + i*D, dOut + i*D, D*sizeof(float), cudaMemcpyDeviceToDevice));
        }
        return dIn;
    }
};


// ---------------------------------------------
// FC Layer
// ---------------------------------------------
class FC : public Layer {
public:
    int input_size, output_size;
    float *d_w=nullptr, *d_b=nullptr;

    // For backward
    float *d_w_grad=nullptr, *d_b_grad=nullptr;
    int inN,inD;

    float *weight_copy = nullptr; // store weights for backward computations

    FC(int in_size, int out_size): input_size(in_size), output_size(out_size) {
        float *h_w=(float*)malloc(in_size*out_size*sizeof(float));
        float *h_b=(float*)malloc(out_size*sizeof(float));
        for(int i=0;i<in_size*out_size;i++) h_w[i]=(float)rand()/RAND_MAX*0.01f;
        for(int i=0;i<out_size;i++) h_b[i]=(float)rand()/RAND_MAX*0.01f;
        CUDA_CALL(cudaMalloc(&d_w,in_size*out_size*sizeof(float)));
        CUDA_CALL(cudaMalloc(&d_b,out_size*sizeof(float)));
        CUDA_CALL(cudaMemcpy(d_w,h_w,in_size*out_size*sizeof(float),cudaMemcpyHostToDevice));
        CUDA_CALL(cudaMemcpy(d_b,h_b,out_size*sizeof(float),cudaMemcpyHostToDevice));
        free(h_w);free(h_b);

        CUDA_CALL(cudaMalloc(&d_w_grad,in_size*out_size*sizeof(float)));
        CUDA_CALL(cudaMalloc(&d_b_grad,out_size*sizeof(float)));
        CUDA_CALL(cudaMemset(d_w_grad,0,in_size*out_size*sizeof(float)));
        CUDA_CALL(cudaMemset(d_b_grad,0,out_size*sizeof(float)));

        isConvOrPool = false;
    }
    ~FC(){
        CUDA_CALL(cudaFree(d_w));
        CUDA_CALL(cudaFree(d_b));
        CUDA_CALL(cudaFree(d_w_grad));
        CUDA_CALL(cudaFree(d_b_grad));
        // if (input_cache) CUDA_CALL(cudaFree(input_cache));
        // if (output_cache) CUDA_CALL(cudaFree(output_cache));
    }

    float* forward(float* input, int N, int D) override {
        inN=N;inD=D;
        if (D!=input_size) {std::cerr<<"FC forward dimension mismatch!\n";}

        float* d_output;
        CUDA_CALL(cudaMalloc(&d_output, N*output_size*sizeof(float)));
        dim3 gridDim(N);
        fcForwardKernel<<<gridDim,1>>>(input,d_w,d_b,d_output,N,input_size,output_size);
        CUDA_CALL(cudaDeviceSynchronize());

        input_cache = input;
        output_cache = d_output;
        this->outN = N; this->outD = output_size;
        return d_output;
    }

    float* backward(float* dOut) override {
        // dOut: (N, output_size)
        // We compute dIn: (N, input_size)
        float *dIn;
        CUDA_CALL(cudaMalloc(&dIn, inN*inD*sizeof(float)));

        // zero out gradients
        CUDA_CALL(cudaMemset(d_w_grad,0,input_size*output_size*sizeof(float)));
        CUDA_CALL(cudaMemset(d_b_grad,0,output_size*sizeof(float)));

        // Compute gradients w.r.t. W and B
        {
            dim3 gridDim(inN);
            fcBackwardKernel<<<gridDim,1>>>(input_cache, dOut, dIn, d_w_grad, d_b_grad, inN, input_size, output_size);
        }

        // Compute gradients w.r.t input
        // Need weights for this
        {
            dim3 gridDim(inN);
            fcBackwardInputKernel<<<gridDim,1>>>(dOut, d_w, dIn, inN, input_size, output_size);
        }

        CUDA_CALL(cudaDeviceSynchronize());
        return dIn;
    }

    void zeroGrad() override {
        CUDA_CALL(cudaMemset(d_w_grad,0,input_size*output_size*sizeof(float)));
        CUDA_CALL(cudaMemset(d_b_grad,0,output_size*sizeof(float)));
    }

    void updateParams(float lr) override {
        int w_size = input_size*output_size;
        thrust::device_ptr<float> w_ptr(d_w);
        thrust::device_ptr<float> dw_ptr(d_w_grad);
        for (int i=0;i<w_size;i++){
            w_ptr[i] -= lr*dw_ptr[i];
        }

        thrust::device_ptr<float> b_ptr(d_b);
        thrust::device_ptr<float> db_ptr(d_b_grad);
        for (int i=0;i<output_size;i++){
            b_ptr[i] -= lr*db_ptr[i];
        }
    }
};


float computeAccuracy(float* d_predictions, float* d_labels, int N, int D) {
    // predictions are (N,D) probabilities after softmax
    // labels are one-hot (N,D)
    // We find argmax of predictions and compare with argmax of labels
    float *h_predictions=(float*)malloc(N*D*sizeof(float));
    float *h_labels=(float*)malloc(N*D*sizeof(float));

    CUDA_CALL(cudaMemcpy(h_predictions,d_predictions,N*D*sizeof(float),cudaMemcpyDeviceToHost));
    CUDA_CALL(cudaMemcpy(h_labels,d_labels,N*D*sizeof(float),cudaMemcpyDeviceToHost));

    int correct=0;
    for (int n=0;n<N;n++){
        int pred_class=0;
        float pred_val=-1.0f;
        int true_class=0;
        for (int d=0; d<D; d++){
            float p=h_predictions[n*D + d];
            if (p>pred_val){
                pred_val=p;
                pred_class=d;
            }
            if (h_labels[n*D + d]==1.0f){
                true_class=d;
            }
        }
        if (pred_class==true_class) correct++;
    }

    free(h_predictions);
    free(h_labels);
    return (float)correct/N;
}

// Softmax (in-place)
void softmax(float* d_data, int N, int D) {
    softmaxKernel<<<N,1>>>(d_data,N,D);
    CUDA_CALL(cudaDeviceSynchronize());
}



// -------------------------------------------------------------------
// Kernel to compute Cross Entropy Loss and its gradient w.r.t predictions
// predictions: (N,D) after softmax
// labels: (N,D) one-hot
// dLoss/dPred: (N,D)
// loss out: scalar (we can store partial sums and reduce)
__global__ void crossEntropyKernel(const float* predictions, const float* labels,
                                   float* dLoss_dPred, float* loss_buffer,
                                   int N, int D) {
    int n = blockIdx.x;
    float loss_val = 0.0f;
    for (int d=0; d<D; d++){
        float y = labels[n*D + d];
        float p = predictions[n*D + d];
        // Avoid log(0) by adding small epsilon if needed
        if(y > 0.0f) {
            loss_val -= logf(p+1e-9f);
        }
        // Gradient w.r.t prediction (P - Y)
        dLoss_dPred[n*D + d] = p - y;
    }
    loss_buffer[n] = loss_val;
}

__global__ void crossEntropyOnlyKernel(const float* pred, const float* lab, float* loss_buf, int N, int D) {
        int n = blockIdx.x;
        float loss_val=0.0f;
        for (int d=0; d<D; d++){
            float y = lab[n*D + d];
            float p = pred[n*D + d];
            if(y>0.0f) loss_val -= logf(p+1e-9f);
        }
        loss_buf[n] = loss_val;
    };


// Function to compute cross-entropy loss and gradient
float crossEntropyLossAndGradient(float* d_predictions, float* d_labels,
                                  float* d_dLoss_dPred, int N, int D) {
    // Allocate buffer for partial losses
    float *d_loss_buffer;
    CUDA_CALL(cudaMalloc(&d_loss_buffer, N*sizeof(float)));

    // Launch kernel
    crossEntropyKernel<<<N,1>>>(d_predictions, d_labels, d_dLoss_dPred, d_loss_buffer, N, D);
    CUDA_CALL(cudaDeviceSynchronize());

    // Copy loss buffer to host and sum up
    float *h_loss_buffer = (float*)malloc(N*sizeof(float));
    CUDA_CALL(cudaMemcpy(h_loss_buffer, d_loss_buffer, N*sizeof(float), cudaMemcpyDeviceToHost));
    float loss=0.0f;
    for (int i=0;i<N;i++){
        loss += h_loss_buffer[i];
    }
    loss /= N; // average over batch

    free(h_loss_buffer);
    CUDA_CALL(cudaFree(d_loss_buffer));

    return loss;
}

// A helper to compute just the cross entropy loss without gradient:
float crossEntropyLossOnly(float* d_predictions, float* d_labels, int N, int D) {
    // Similar to above but no gradient needed:
    float *d_loss_buffer;
    CUDA_CALL(cudaMalloc(&d_loss_buffer, N*sizeof(float)));

    // Use a modified kernel that doesn't compute gradients:
    // For simplicity, we can just reuse the existing kernel by passing in NULL for dLoss_dPred
    // but we must ensure that kernel checks for NULL. Let's just reimplement a simple kernel here.

    crossEntropyOnlyKernel<<<N,1>>>(d_predictions, d_labels, d_loss_buffer, N, D);
    CUDA_CALL(cudaDeviceSynchronize());

    float *h_loss_buffer = (float*)malloc(N*sizeof(float));
    CUDA_CALL(cudaMemcpy(h_loss_buffer, d_loss_buffer, N*sizeof(float), cudaMemcpyDeviceToHost));
    float loss=0.0f;
    for (int i=0;i<N;i++){
        loss+=h_loss_buffer[i];
    }
    loss/=N;

    free(h_loss_buffer);
    CUDA_CALL(cudaFree(d_loss_buffer));
    return loss;
}



// ---------------------------------------------
// Model Class
// ---------------------------------------------
class Model {
public:
    std::vector<Layer*> layers;

    void addLayer(Layer* layer) {
        layers.push_back(layer);
    }

    float* forward(float* input, int N, int H, int W, int C) {
        int curN=N, curH=H, curW=W, curC=C, curD=0;
        float* current_data = input;
        for (auto l : layers) {
            if (l->isConvOrPool) {
                current_data = l->forward(current_data, curN, curH, curW, curC);
                curN = l->outN; curH = l->outH; curW = l->outW; curC = l->outC;
                curD = 0;
            } else {
                if (dynamic_cast<Flatten*>(l)) {
                    current_data = l->forward(current_data, curN, curH, curW, curC);
                    curN = l->outN; curD = l->outD;
                    curH=0; curW=0; curC=0;
                } else {
                    current_data = l->forward(current_data, curN, curD);
                    curN = l->outN; curD = l->outD;
                }
            }
        }
        return current_data;
    }

    float* backward(float* dLoss_dOut) {
        // dLoss_dOut is gradient of loss w.r.t final output of model
        // We propagate backwards through layers
        float* current_grad = dLoss_dOut;
        for (int i=(int)layers.size()-1; i>=0; i--) {
            current_grad = layers[i]->backward(current_grad);
        }
        return current_grad; // this is dLoss/dInput of the entire model
    }

    void zeroGrad() {
        for (auto l: layers) {
            l->zeroGrad();
        }
    }

    void updateParams(float lr) {
        for (auto l: layers) {
            l->updateParams(lr);
        }
    }
};

int main() {
    int num_classes = 10;
    int train_size = 10000;   // total training samples
    int test_size = 1000;     // total test samples
    int N=5000;
    int epochs = 10;
    float learning_rate = 0.00001f;
    int H=28, W=28, C=1;

    std::cout << "Beginning Program!!!!!!" << std::endl;

    int img_rows, img_cols;
    float* h_train_images = (float*)malloc(train_size * H * W * C * sizeof(float));
    readMNISTImages("mnist-train-images", h_train_images, train_size, img_rows, img_cols);
    std::cout << "Read training images file in." << std::endl;

    float* h_train_labels = (float*)malloc(train_size*num_classes*sizeof(float));
    readMNISTLabels("mnist-train-labels", h_train_labels, train_size, num_classes);
    std::cout << "Read training labels file in." << std::endl;


    float* h_test_images = (float*)malloc(test_size * H * W * C * sizeof(float));
    readMNISTImages("mnist-test-images", h_test_images, test_size, img_rows, img_cols);
    std::cout << "Read testing images file in." << std::endl;

    float* h_test_labels = (float*)malloc(test_size*num_classes*sizeof(float));
    readMNISTLabels("mnist-test-labels", h_test_labels, test_size, num_classes);
    std::cout << "Read testing labels file in." << std::endl;


    // Copy entire dataset to GPU (in practice you might do this batch-by-batch)
    float *d_train_input;
    float *d_train_labels;
    cudaMalloc(&d_train_input, train_size*H*W*C*sizeof(float));
    cudaMalloc(&d_train_labels, train_size*num_classes*sizeof(float));
    cudaMemcpy(d_train_input,h_train_images,train_size*H*W*C*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_train_labels,h_train_labels,train_size*num_classes*sizeof(float),cudaMemcpyHostToDevice);

    float *d_test_input,*d_test_labels;
    cudaMalloc(&d_test_input,test_size*H*W*C*sizeof(float));
    cudaMalloc(&d_test_labels,test_size*num_classes*sizeof(float));
    cudaMemcpy(d_test_input,h_test_images,test_size*H*W*C*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_test_labels,h_test_labels,test_size*num_classes*sizeof(float),cudaMemcpyHostToDevice);


    // Build model
    Model model;
    model.addLayer(new Conv2D(3,1,30));    // (N,28,28,1) -> (N,26,26,30)
    model.addLayer(new MaxPool(2));        // (N,26,26,30) -> (N,13,13,30)
    model.addLayer(new Flatten());         // (N,13,13,30) -> (N,5070)
    model.addLayer(new FC(13*13*30,10));  // (N,6750) -> (N,10)

    std::cout << "Created model. " << std::endl;

    int batch_size = N; // from above N=2
    int steps_per_epoch = train_size / batch_size; // integer division

    std::cout << "===== Beginning Training w/ training_set_size: " << train_size <<
      "  batch_size: " << batch_size << "  epochs: " << epochs <<
      "  steps/epoch: " << steps_per_epoch << "  learning_rate: " << learning_rate << " =====" << std::endl;
    for (int epoch=0; epoch<epochs; epoch++){
        std::cout << "Starting Epoch " << (epoch + 1) << std::endl;

        float epoch_loss=0.0f;
        std::cout << "\t";

        // Training loop over batches
        for (int step=0; step<steps_per_epoch; step++){
            int start_idx = step*batch_size;
            // Pointers to this batch
            float *d_batch_input = d_train_input + start_idx*(H*W*C);
            float *d_batch_labels = d_train_labels + start_idx*(num_classes);

            // Forward pass
            float *d_out = model.forward(d_batch_input, batch_size, H, W, C);
            softmax(d_out,batch_size,num_classes);

            // Compute loss and gradient for this batch
            float *d_dLoss_dOut;
            cudaMalloc(&d_dLoss_dOut,batch_size*num_classes*sizeof(float));
            float batch_loss = crossEntropyLossAndGradient(d_out, d_batch_labels, d_dLoss_dOut, batch_size, num_classes);
            epoch_loss += batch_loss;

            // Backprop
            model.zeroGrad();
            float *d_dLoss_dIn = model.backward(d_dLoss_dOut);

            // Update parameters
            model.updateParams(learning_rate);

            // Cleanup this batch
            cudaFree(d_out);
            cudaFree(d_dLoss_dOut);
            cudaFree(d_dLoss_dIn);

            std::cout << "*";
        }

        std::cout << std::endl;

        // Average loss over the epoch
        epoch_loss /= steps_per_epoch;

        // Evaluate on test set
        float *d_test_out = model.forward(d_test_input,test_size,H,W,C);
        softmax(d_test_out,test_size,num_classes);
        float test_loss = crossEntropyLossOnly(d_test_out,d_test_labels,test_size,num_classes);
        float test_acc = computeAccuracy(d_test_out,d_test_labels,test_size,num_classes);

        std::cout << "\tEpoch "<<(epoch+1)<<" Train Loss: "<<epoch_loss<<" | Test Loss: "<<test_loss<<" | Test Acc: "<<test_acc<<"\n";

        cudaFree(d_test_out);
    }

    // Cleanup
    free(h_train_images); free(h_train_labels);
    free(h_test_images); free(h_test_labels);
    cudaFree(d_train_input);
    cudaFree(d_train_labels);
    cudaFree(d_test_input);
    cudaFree(d_test_labels);

    for (auto l : model.layers) delete l;

    return 0;
}


Beginning Program!!!!!!
Read training images file in.
Read training labels file in.
Read testing images file in.
Read testing labels file in.
Created model. 
===== Beginning Training w/ training_set_size: 10000  batch_size: 5000  epochs: 10  steps/epoch: 2  learning_rate: 1e-05 =====
Starting Epoch 1
	**
	Epoch 1 Train Loss: 6.07335 | Test Loss: 9.75133 | Test Acc: 0.334
Starting Epoch 2
	**
	Epoch 2 Train Loss: 9.11766 | Test Loss: 11.2496 | Test Acc: 0.3
Starting Epoch 3
	**
	Epoch 3 Train Loss: 11.2114 | Test Loss: 14.1091 | Test Acc: 0.283
Starting Epoch 4
	**
	Epoch 4 Train Loss: 12.9466 | Test Loss: 13.2288 | Test Acc: 0.32
Starting Epoch 5
	**
	Epoch 5 Train Loss: 12.2174 | Test Loss: 11.2092 | Test Acc: 0.339
Starting Epoch 6
	**
	Epoch 6 Train Loss: 8.65808 | Test Loss: 5.33045 | Test Acc: 0.607
Starting Epoch 7
	**
	Epoch 7 Train Loss: 3.51438 | Test Loss: 2.05589 | Test Acc: 0.757
Starting Epoch 8
	**
	Epoch 8 Train Loss: 1.68532 | Test Loss: 1.89768 | Test Acc: 0.762
Starti